In [1]:
# This is the children's story "Hoppelpoppel - wo bist du" by my most favourite author from the last years, Hans Fallada
# as a preparation for a visualization in Tableau.

import pandas as pd
import numpy as np
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from itertools import chain
import re

In [2]:
# Scraping story text

my_url = 'https://www.projekt-gutenberg.org/fallada/hoppelpo/chap001.html'

uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, 'html.parser')

In [3]:
container = page_soup.find_all('p')
container = container[0]

In [4]:
title = container.find_all('h3')
title = str(title[0])

In [5]:
story = container.find_all('p')
paragraphs = []
for i in story:
    paragraphs.append(str(i))

In [6]:
for i in range(len(paragraphs)):
    paragraphs[i] = paragraphs[i].replace('<p>', '')
    paragraphs[i] = paragraphs[i].replace('</p>', '') 

In [7]:
words = []
try:
    for i in range(len(paragraphs)):
        for j in range(len(paragraphs[i])):
            words.append(paragraphs[j].split(' ')) 
except:
    pass

In [8]:
# Flatten nested lists in words

words = list(chain.from_iterable(words))

In [9]:
for i in range(len(words)):
        words[i] = words[i].strip()
        words[i] = words[i].replace('\xa0...', '')
        words[i] = words[i].replace('<a', '')
        words[i] = words[i].replace('</a>', '')
        words[i] = words[i].replace('\n', '')
        words[i] = words[i].replace('-', '')
        words[i] = words[i].replace('–', '')
        words[i] = words[i].replace('.', '')
        words[i] = words[i].replace(':', '')
        words[i] = words[i].replace(',', '')
        words[i] = words[i].replace('>', '')
        words[i] = words[i].replace('<', '')
        words[i] = words[i].replace('›', '')
        words[i] = words[i].replace('‹', '')
        words[i] = words[i].replace('?', '')
        words[i] = words[i].replace('!', '')
        words[i] = words[i].replace('«', '')
        words[i] = words[i].replace('»', '')
        words[i] = words[i].replace('(', '')
        words[i] = words[i].replace(')', '')
        words[i] = words[i].replace(' ', '')
        words[i] = re.sub(str('id='+'"'+'page\d'+'"'), '', words[i])
        words[i] = re.sub(str('name='+'"'+'page\d'+'"'), '', words[i])

In [10]:
# Remove empty strings in words

while("" in words) :
    words.remove("")

In [11]:
df = pd.DataFrame(words)

In [12]:
df.reset_index(inplace = True)

In [13]:
#df.columns = ['rank', 'text']
df.columns = ['rank', 'text']
df['text2'] = ''

In [14]:
# Cleaning last HTML code

part1 = df[df.text == 'title="Dieter7/lac"']
part2 = df[df.text == 'title="suru/Dieter7"']
part3 = df[df.text == 'title="lac/Dieter7"']
eraser = pd.concat([part1, part2, part3])

erank = []
for i in range(len(eraser)):
    erank.append(int(eraser.index[i]))

for i in erank:
    df = df.drop([i])

In [15]:
df.reset_index(drop = True, inplace = True)

In [16]:
df = df.drop('rank', axis = 1)

In [17]:
df.reset_index(inplace = True)

In [18]:
df.columns = ['rank', 'text', 'text2']

In [19]:
# Fill 'text2' with text from previous row

for i in range(len(df)-1):
    if i < len(df):
        try:
            df['text2'][i] = df.text[i+1] 
        except:
            pass

<ipython-input-19-d31fb755fdcd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text2'][i] = df.text[i+1]


In [20]:
df['text3_lowercase'] = df.text.str.lower()

In [21]:
# Setting for displaying the full dataframe 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [22]:
# Creating a dataframe with German words in lowercase for count of unique values

df2 = pd.Series(df.text3_lowercase.unique())
df2 = pd.DataFrame(df2)

In [23]:
df2.columns = ['text_unique']
df2['count_text'] = ''

In [24]:
# Counting the unique words

for i in range(len(df2)):
    x = df2.text_unique[i]      # x stores each single word in text_unique
    df2['count_text'][i] = df[df.text3_lowercase == x].text3_lowercase.count()

In [25]:
df['count_text'] = ''
df['lat'] = ''
df['lon'] = ''
df['path_ID'] = ''
df['path_start_end'] = ''

In [26]:
for i in range(len(df2)):
    x = df2.text_unique[i]
    for j in range(len(df)):
        if df.text3_lowercase[j] == x:
            df['count_text'][j] = df2.count_text[i] 

<ipython-input-26-07debcdcef62>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count_text'][j] = df2.count_text[i]


In [27]:
df2.shape

(486, 2)

In [28]:
df.to_csv('fallada_hoppelpoppel.csv')